In [149]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
from itertools import chain, combinations

First let's process the data.

In [150]:
with open('car_data.txt') as f:
    rows = [line.split() for line in f]

In [151]:
df = pd.DataFrame.from_records(rows, columns = ['make/model', 'vol', 'hp', 'mpg', 'sp', 'wt'])

In [152]:
df['vol'] = df['vol'].astype('int')

In [153]:
df['hp'] = df['hp'].astype('int')

In [154]:
df['mpg'] = df['mpg'].astype('float')

In [155]:
df['sp'] = df['sp'].astype('int')

In [156]:
df['wt'] = df['wt'].astype('float')

In [157]:
df

,make/model,vol,hp,mpg,sp,wt
0,GM/GeoMetroXF1,89,49,65.4,96,17.5
1,GM/GeoMetro,92,55,56.0,97,20.0
2,GM/GeoMetroLSI,92,55,55.9,97,20.0
3,SuzukiSwift,92,70,49.0,105,20.0
4,DaihatsuCharade,92,53,46.5,96,20.0
...,...,...,...,...,...,...
77,Mercedes500SL,50,322,18.1,165,45.0
78,Mercedes560SEL,115,238,17.2,140,45.0
79,JaguarXJSConvert,50,263,17.0,147,45.0
80,BMW750IL,119,295,16.7,157,45.0


# Part (a)

In [158]:
Y = df['mpg'].to_numpy().reshape(-1,1)

In [159]:
X = df[['vol', 'hp', 'sp', 'wt']].to_numpy()

In [160]:
X = np.concatenate((np.ones((len(X), 1)), X), axis=1)

In [161]:
beta = inv(X.T @ X) @ X.T @ Y

In [162]:
Y_hat = X @ beta

In [163]:
# The array of residuals.
res = Y - Y_hat

In [164]:
mse = (res**2).mean()

In [165]:
sigma = np.sqrt((res**2).sum() / (len(res) - len(beta)))

In [166]:
to_print = map(str, beta.flatten().tolist())

In [167]:
print('beta: ' + ' '.join(to_print))
print(f'Mean squared error: {mse:.4f}')

beta: 192.43775332063407 -0.015645011317609703 0.39221231455809424 -1.2948184772442466 -1.8598037272929624
Mean squared error: 12.5290


We see that 'sp' (top speed) and 'wt' (weight) are strongly negatively correlated with 'mpg.' Additionally, the mean squared error is much less than we saw when fitting a simple linear regression from 'hp.'

# Part (b)

In [168]:
variables = ['vol', 'hp', 'sp', 'wt']

In [169]:
# An array to store the variables with the lowest C_p score
# for the current round of forward stepwise selection.
cur_best_vars = []

In [170]:
# An array to store the variables with the lowest C_p score
# among all the rounds of stepwise selection.
best_vars = []

In [171]:
# The lowest C_p score among all rounds of stepwise selection.
best_mallow = float('inf')

In [172]:
for i in range(4):
    # The lowest C_p score for the current round and the corresponding variable.
    cur_best_mallow = float('inf')
    cur_best_var = None
    
    for var in variables:
        # Try adding on var to cur_best_vars and evaluate the C_p score.
        the_vars = cur_best_vars + [var]
        X = df[the_vars].to_numpy()
        X = np.concatenate((np.ones((len(X), 1)), X), axis=1)
        beta = inv(X.T @ X) @ X.T @ Y
        Y_hat = X @ beta
        
        # Calculate the C_p score.
        R_tr = np.sum((Y - Y_hat)**2)
        mallow = R_tr + 2 * len(the_vars) * sigma**2
        print(f'{the_vars}, C_p score: {mallow:.4f}')
        
        # Store this variable and its C_p score
        # if the C_p score is the best this round.
        if mallow < cur_best_mallow:
            cur_best_var = var
            cur_best_mallow = mallow
            
    cur_best_vars = cur_best_vars + [cur_best_var]
    
    # Reset best_mallow and cur_best_mallow if the best
    # C_p score dropped this round. Otherwise break.
    if cur_best_mallow < best_mallow:
        best_mallow = cur_best_mallow
        best_vars = cur_best_vars
    else:
        break
    
    variables.remove(cur_best_var)
    print('')

['vol'], C_p score: 7032.5378
['hp'], C_p score: 3076.1203
['sp'], C_p score: 4291.5494
['wt'], C_p score: 1492.6869

['wt', 'vol'], C_p score: 1515.4894
['wt', 'hp'], C_p score: 1483.9923
['wt', 'sp'], C_p score: 1436.3913

['wt', 'sp', 'vol'], C_p score: 1417.1098
['wt', 'sp', 'hp'], C_p score: 1113.7056

['wt', 'sp', 'hp', 'vol'], C_p score: 1134.1224


In [173]:
print(f'Best variables for forward stepwise: {best_vars}')
print(f'Best C_p score: {best_mallow:.4f}')

Best variables for forward stepwise: ['wt', 'sp', 'hp']
Best C_p score: 1113.7056


Now we'll repeat the analysis for backward stepwise selection.

In [174]:
variables = ['vol', 'hp', 'sp', 'wt']

In [175]:
# An array to store the variables with the lowest C_p score
# for the current round of backward stepwise selection.
cur_best_vars = variables[:]

In [176]:
# An array to store the variables with the lowest C_p score
# among all the rounds of stepwise selection.
best_vars = variables[:]

In [177]:
# Calculate the C_p score with all variables.
X = df[cur_best_vars].to_numpy()
X = np.concatenate((np.ones((len(X), 1)), X), axis=1)
beta = inv(X.T @ X) @ X.T @ Y
Y_hat = X @ beta
R_tr = np.sum((Y - Y_hat)**2)
best_mallow = R_tr + 2 * len(cur_best_vars) * sigma**2

In [178]:
print(f'{cur_best_vars}, C_p score: {best_mallow:.4f}')

['vol', 'hp', 'sp', 'wt'], C_p score: 1134.1224


In [179]:
for i in range(3):
    # The lowest C_p score for the current round and the corresponding variable.
    cur_best_mallow = float('inf')
    cur_best_var = None
    
    for var in variables:
        # Try removing var from cur_best_vars and evaluate the C_p score.
        the_vars = cur_best_vars[:]
        the_vars.remove(var)
        X = df[the_vars].to_numpy()
        X = np.concatenate((np.ones((len(X), 1)), X), axis=1)
        beta = inv(X.T @ X) @ X.T @ Y
        
        # Calculate the C_p score.
        Y_hat = X @ beta
        R_tr = np.sum((Y - Y_hat)**2)
        mallow = R_tr + 2 * len(the_vars) * sigma**2
        print(f'{the_vars}, C_p score: {mallow:.4f}')

        # Store this variable and its C_p score
        # if the C_p score is the best this round.
        if mallow < cur_best_mallow:
            cur_best_var = var
            cur_best_mallow = mallow
    
    # Reset best_mallow and cur_best_mallow if the best
    # C_p score dropped this round. Otherwise break.
    if cur_best_mallow < best_mallow:
        cur_best_vars.remove(cur_best_var)
        best_mallow = cur_best_mallow
        best_vars = cur_best_vars
        
        variables.remove(cur_best_var)
        print('')
    else:
        break

['hp', 'sp', 'wt'], C_p score: 1113.7056
['vol', 'sp', 'wt'], C_p score: 1417.1098
['vol', 'hp', 'wt'], C_p score: 1480.7991
['vol', 'hp', 'sp'], C_p score: 2121.2013

['sp', 'wt'], C_p score: 1436.3913
['hp', 'wt'], C_p score: 1483.9923
['hp', 'sp'], C_p score: 2409.8936


In [180]:
print(f'Best variables for backward stepwise: {best_vars}')
print(f'Best C_p score: {best_mallow:.4f}')

Best variables for backward stepwise: ['hp', 'sp', 'wt']
Best C_p score: 1113.7056


We find that backward and forward stepwise selection both yield the same set of variables: 'hp', 'sp', and 'wt.'

Part (c)
--

In [220]:
variables = np.array(['vol', 'hp', 'sp', 'wt'])

In [221]:
X = df[variables].to_numpy()

In [222]:
X = np.concatenate((np.ones((len(X), 1)), X), axis=1)

In [223]:
n = X.shape[0]

In [224]:
beta = inv(X.T @ X) @ X.T @ Y

In [225]:
Y_hat = X @ beta

In [226]:
# The array of residuals.
res = Y - Y_hat

In [227]:
# Unbiased estimator of sigma.
sigma = np.sqrt((res**2).sum() / (len(res) - len(beta)))

In [228]:
# The estimated standard errors for the coefficients of beta
# are the diagonal entries of the matrix se calculated below.
se = sigma**2 * inv(X.T @ X)

In [229]:
# The Wald statistics for the coefficients of beta.
Wald = beta.flatten() / se.diagonal()

In [230]:
# Drop the Wald coefficient for beta_0
Wald = Wald[1:]
Wald = list(enumerate(Wald))

In [231]:
Wald.sort(key=lambda x: abs(x[1]), reverse=True)

In [232]:
print(f'Ranked Wald statistics: {Wald}')

Ranked Wald statistics: [(1, 59.17530990583489), (3, -40.8534551345469), (0, -30.029219991271308), (2, -21.61126177566326)]


In [233]:
# An array to store the Zheng-Loh values RSS + j*sigma^2*log(n)
Zheng_Loh = []

In [234]:
for j in range(1,5):
    ind = [Wald[i][0] for i in range(j)]
    X_ = X[:, ind]
    beta_ = inv(X_.T @ X_) @ X_.T @ Y
    Y_hat_ = X_ @ beta_
    res = Y - Y_hat_
    rss = (Y - Y_hat_).sum()
    zl = rss + j * sigma**2 * np.log(n)
    Zheng_Loh.append(zl)

In [235]:
print(f'Zheng-Loh values: {Zheng_Loh}')

Zheng-Loh values: [253.83497919601743, 216.85399769875423, 176.39149597798055, 235.18866130426315]


In [236]:
j_hat = np.argmin(Zheng_Loh) + 1

In [238]:
best_vars = variables[[Wald[i][0] for i in range(j_hat)]]

In [240]:
print(f'Best variables for backward stepwise: {best_vars}')
print(f'Best Zheng-Loh score: {Zheng_Loh[j_hat-1]:.4f}')

Best variables for backward stepwise: ['hp' 'wt' 'vol']
Best Zheng-Loh score: 176.3915


The best variables identified by Zheng-Loh are a bit different: 'sp' was swapped out for 'vol.'

Part (d)
--

In [271]:
variables = np.array(['vol', 'hp', 'sp', 'wt'])

In [272]:
n = len(Y)

In [273]:
# A function to get the power set of the set of variables.
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [274]:
# Ignore the subset with no variables.
subsets = list(powerset(variables))[1:]

In [275]:
best_mallow = float('inf')

In [276]:
best_mallow_vars = None

In [277]:
best_bic = -float('inf')

In [278]:
best_bic_vars = None

In [279]:
for subset in subsets:
    X = df[list(subset)].to_numpy()
    X = np.concatenate((np.ones((len(X), 1)), X), axis=1)
    beta = inv(X.T @ X) @ X.T @ Y
    Y_hat = X @ beta
    R_tr = np.sum((Y - Y_hat)**2)
    mallow = R_tr + 2 * len(subset) * sigma**2
    if mallow < best_mallow:
        best_mallow = mallow
        best_mallow_vars = subset
    log_lik = -n * np.log(sigma) - (1 / (2 * sigma**2))*R_tr
    bic = log_lik - (len(subset)/2)*np.log(n)
    if bic > best_bic:
        best_bic = bic
        best_bic_vars = subset
    print(f'Subset {subset} --- BIC: {bic:.4f}, C_p: {mallow:.4f}')

Subset ('vol',) --- BIC: -370.9695, C_p: 7032.5378
Subset ('hp',) --- BIC: -222.7071, C_p: 3076.1203
Subset ('sp',) --- BIC: -268.2540, C_p: 4291.5494
Subset ('wt',) --- BIC: -163.3697, C_p: 1492.6869
Subset ('vol', 'hp') --- BIC: -195.8806, C_p: 2328.1379
Subset ('vol', 'sp') --- BIC: -222.1789, C_p: 3029.9136
Subset ('vol', 'wt') --- BIC: -165.4275, C_p: 1515.4894
Subset ('hp', 'sp') --- BIC: -198.9443, C_p: 2409.8936
Subset ('hp', 'wt') --- BIC: -164.2472, C_p: 1483.9923
Subset ('sp', 'wt') --- BIC: -162.4634, C_p: 1436.3913
Subset ('vol', 'hp', 'sp') --- BIC: -189.3293, C_p: 2121.2013
Subset ('vol', 'hp', 'wt') --- BIC: -165.3309, C_p: 1480.7991
Subset ('vol', 'sp', 'wt') --- BIC: -162.9442, C_p: 1417.1098
Subset ('hp', 'sp', 'wt') --- BIC: -151.5745, C_p: 1113.7056
Subset ('vol', 'hp', 'sp', 'wt') --- BIC: -153.5429, C_p: 1134.1224


In [281]:
print(f'Best subset for BIC: {best_bic_vars}')
print(f'Best BIC score: {best_bic:.4f}')
print(f'Best subset for Mallow C_p: {best_mallow_vars}')
print(f'Best Mallow C_p score: {best_mallow:.4f}')

Best subset for BIC: ('hp', 'sp', 'wt')
Best BIC score: -151.5745
Best subset for Mallow C_p: ('hp', 'sp', 'wt')
Best Mallow C_p score: 1113.7056


We see that BIC and $C_p$ both identify the same subset of variables, which were previously identified by both forward and backward stepwise selection with $C_p$.